# 400 - gradient et backward

On peut avoir un réseau de neurones comme un gros sandwitch avec de multiples couches de neurones dans lequel on a envie d'insérer sa propre couche mais pour cela il faut interférer avec le gradient. J'ai repris le tutoriel [pytorch: defining new autograd functions](https://pytorch.org/tutorials/beginner/examples_autograd/two_layer_net_custom_function.html). L'exemple suivant [Extending Torch](https://pytorch.org/docs/master/notes/extending.html). J'aimais bien l'API de la version 0.4 mais je ne la trouve plus sur internet. Elle laissait sans doute un peu trop de liberté.

In [1]:
import time
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
print("torch", torch.__version__)
from torchvision import datasets, transforms
from tqdm import tqdm

torch 1.0.1


In [2]:
%matplotlib inline

In [3]:
BATCH_SIZE = 64
TEST_BATCH_SIZE = 64
DATA_DIR = 'data/'
USE_CUDA = False  # switch to True if you have GPU
N_EPOCHS = 2 # 5

In [4]:
from jyquickhelper import add_notebook_menu
add_notebook_menu()

## Couche personnalisée

L'exemple suivant illustre comment définir une couche intermédiaire qui obéit à ses propres règles. Elle doit implémenter les deux méthodes ``forward`` qui calcule la prédiction pour la couche suivante et ``backward`` pour le calcul du gradient pour la couche précédente. Il reste la variable ``ctx`` qui permet de stocker des informations qui persistent jusqu'au calcul du gradient.

In [5]:
class MyReLU(torch.autograd.Function):

    @staticmethod
    def forward(ctx, input):
        ctx.save_for_backward(input)
        return input.clamp(min=0)

    @staticmethod
    def backward(ctx, grad_output):
        input, = ctx.saved_tensors
        grad_input = grad_output.clone()
        grad_input[input < 0] = 0
        return grad_input

## exemple avec MNIST

C'est à ce moment que je choisis pour bifurque sur un autre tutoriel [pytorch et MNIST](https://pytorch.org/tutorials/beginner/blitz/neural_networks_tutorial.html) qui commence sur un réseau de neurones pour *MNIST*.

In [7]:
dtype = torch.float
device = torch.device("cpu")

In [8]:
import torch
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

    def num_flat_features(self, x):
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features


net = Net()
print(net)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


## Le gradient

On choisit une entrée aléatoire mais aux mêmes dimensions qu'une image et on calcule la sortie du réseau de neurones.

In [9]:
input = torch.randn(1, 1, 32, 32)
out = net(input)
print(out)

tensor([[ 0.0287,  0.0559, -0.0561, -0.0430, -0.0370, -0.0889, -0.1406, -0.0248,
          0.0059,  0.0725]], grad_fn=<AddmmBackward>)


Pour calculer le gradient, il faut choisir une erreur... On prend le [MSELoss](https://pytorch.org/docs/stable/nn.html?highlight=mseloss#torch.nn.MSELoss) même si ça n'a rien à voir avec le problème initial mais on s'en fout.

In [10]:
criterion = nn.MSELoss()

On choisit une entrée aléatoire...

In [11]:
target = torch.randn(10)

On calcule une sortie et on redimensionne la cible de sorte qu'elle est la même dimension que la cible.

In [12]:
output = net(input)
target = target.view(1, -1)  # make it the same shape as output
output, target

(tensor([[ 0.0287,  0.0559, -0.0561, -0.0430, -0.0370, -0.0889, -0.1406, -0.0248,
           0.0059,  0.0725]], grad_fn=<AddmmBackward>),
 tensor([[ 2.3820, -1.2810,  0.5971, -0.9040,  0.5829,  0.8919,  0.0057, -0.7593,
           0.3045,  0.1073]]))

Maintenantn on calcule l'erreur de prédiction...

In [13]:
loss = criterion(output, target)
loss

tensor(1.0491, grad_fn=<MseLossBackward>)

Et on calcule enfin le gradient :

In [14]:
loss.backward()

Je m'attendais à trouver le gradient sur la forme d'un beau vecteur mais comme c'est un réseau de neurones, le gradient est stocké dans chacune des couches sous la forme d'un gradient et encore en deux morceaux...

In [15]:
net.conv1.bias.grad

tensor([ 0.0077, -0.0058,  0.0442, -0.0226,  0.0169, -0.0129])

In [16]:
net.conv1.weight.grad

tensor([[[[ 7.9754e-04,  9.6428e-03, -2.8664e-03,  2.6580e-03,  6.4314e-03],
          [ 6.3015e-03, -2.9178e-03, -1.2980e-02,  1.3888e-02, -1.1764e-02],
          [-2.1683e-03,  1.4615e-02,  1.4769e-02, -6.1542e-03, -1.6383e-02],
          [-1.5124e-02, -2.2362e-03,  2.2944e-03,  1.6645e-03,  7.3576e-03],
          [-5.4342e-03, -7.8502e-03, -8.6421e-03, -2.4906e-03, -5.4626e-03]]],


        [[[ 1.8100e-02,  7.8166e-03,  2.8443e-02, -3.9466e-03,  2.4189e-02],
          [-8.7754e-03, -1.0982e-02,  1.9052e-02,  1.4167e-02,  3.0474e-02],
          [ 1.6203e-04, -1.2932e-02,  4.1938e-03,  4.6741e-03, -1.3816e-02],
          [ 1.6534e-03, -1.5473e-02, -1.0026e-03, -2.6415e-02, -1.9753e-03],
          [-8.6612e-03, -2.3701e-02, -1.8352e-02, -1.2497e-02,  3.3016e-03]]],


        [[[-1.7104e-02,  1.8906e-02, -3.1089e-03, -2.9519e-02, -1.7429e-02],
          [-3.1063e-02,  2.2333e-02, -1.5788e-02, -4.9390e-04, -4.5373e-03],
          [ 2.4545e-03, -1.2397e-03, -1.0653e-02, -1.5522e-02, -3.45

## On change de couche

Non, il n'est pas trois heures du matin et personne ne pleure.

In [17]:
class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(-1, self.num_flat_features(x))
        y = F.relu(self.fc1(x))
        x = MyReLU(self.fc1(x))
        x = MyReLU(self.fc2(x))
        x = self.fc3(x)
        return x

    def num_flat_features(self, x):
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features


net = Net()
print(net)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


Evidemment, ça ne marche jamais du premier coup. On a beaucoup sous-estimé le temps de préparation des cours avec l'informatique. Il est seulement 9h du soir, l'humilité vient seulement après minuit.

In [18]:
try:
    output = net(input)
except Exception as e:
    print(e)

'MyReLU' object has no attribute 'dim'


J'avoue que j'ai tout lu en diagonal pensant que ma science infinie me permettrait de combler mon ignorance.

In [19]:
target = target.view(1, -1)  # make it the same shape as output
output, target

(tensor([[ 0.0287,  0.0559, -0.0561, -0.0430, -0.0370, -0.0889, -0.1406, -0.0248,
           0.0059,  0.0725]], grad_fn=<AddmmBackward>),
 tensor([[ 2.3820, -1.2810,  0.5971, -0.9040,  0.5829,  0.8919,  0.0057, -0.7593,
           0.3045,  0.1073]]))

Honnêtement, il est temps de dîner !

## After the bug...